# Capstone 3 - Book Recommendation System
Sudha Kumar
2/04/2021

# Documentation

Documentation is the sixth step in the Data Science Method.  The following will be performed in this step:

1. Review the Results
2. Finalize Code
3. Finalize Documentation
4. Create a Project Report
5. Create a Slide Deck for the Executive Audience

In [44]:
#load python packages
import os
import pandas as pd
import pandas.api.types as ptypes
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')

# Best Model - SVD

In [45]:
df = pd.read_csv("../Data_Wrangle_EDA/data/Cap3_step23_output.csv")
df.head()

,Unnamed: 0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age
0,13715,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,85526.0,0.0,"victoria, british columbia, canada",36.0
1,13716,0804106304,The Joy Luck Club,Amy Tan,1994,Prentice Hall (K-12),85526.0,0.0,"victoria, british columbia, canada",36.0
2,13717,0786868716,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,85526.0,0.0,"victoria, british columbia, canada",36.0
3,13718,0060929790,One Hundred Years of Solitude,Gabriel Garcia Marquez,1998,Perennial,85526.0,0.0,"victoria, british columbia, canada",36.0
4,13719,0452282152,Girl with a Pearl Earring,Tracy Chevalier,2001,Plume Books,85526.0,7.0,"victoria, british columbia, canada",36.0


In [46]:
df = df.drop(["Unnamed: 0"], axis=1)

In [47]:
df.shape

(429486, 9)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429486 entries, 0 to 429485
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 429486 non-null  object 
 1   Book-Title           429486 non-null  object 
 2   Book-Author          429486 non-null  object 
 3   Year-Of-Publication  429486 non-null  int64  
 4   Publisher            429486 non-null  object 
 5   User-ID              429486 non-null  float64
 6   Book-Rating          429486 non-null  float64
 7   Location             429486 non-null  object 
 8   Age                  429486 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 29.5+ MB


In [49]:
df_reviews = df[['User-ID','ISBN','Book-Rating']]
df_reviews.head()

,User-ID,ISBN,Book-Rating
0,85526.0,0002005018,0.0
1,85526.0,0804106304,0.0
2,85526.0,0786868716,0.0
3,85526.0,0060929790,0.0
4,85526.0,0452282152,7.0


In [50]:
df_reviews.shape

(429486, 3)

In [51]:
df_sample = df_reviews.sample(n=100000, random_state=1)
df_sample.rename(columns = {'User-ID' : 'userID', 'ISBN' : 'itemID', 'Book-Rating' : 'rating'}, inplace=True)
df_sample.head()

,userID,itemID,rating
124756,13093.0,0345441109,0.0
261251,105374.0,0345404114,0.0
11937,230522.0,0590407201,10.0
271158,259829.0,0886771528,0.0
220988,222050.0,0486272842,10.0


In [52]:
df_sample['bookId']=pd.factorize(df_sample['itemID'].tolist())[0]

In [89]:
df_sample.nunique()

userID     1279
itemID    59338
rating       11
bookId    59338
dtype: int64

In [90]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_sample[['userID', 'bookId', 'rating']], reader)

#algo_SVD = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
algo_SVD = SVD()

#trainset = data.build_full_trainset()
#algo_SVD.fit(trainset)

#testset = trainset.build_anti_testset()
#predictions = algo_SVD.test(testset)

trainset, testset = train_test_split(data, test_size=.3)
algo_SVD.fit(trainset)

predictions = algo_SVD.test(testset)

# Compute and print Root Mean Squared Error
accuracy.rmse(predictions, verbose=True)

RMSE: 3.2111


3.2111211262724324

In [91]:
algo_SVD.predict(uid = 7346.0, iid = 19409, r_ui = 8.0)

Prediction(uid=7346.0, iid=19409, r_ui=8.0, est=8.133611790244698, details={'was_impossible': False})

# Top 5 predictions for users in testset

In [92]:
from collections import defaultdict

n = 5

# First map the predictions to each user.
top_n = defaultdict(list)
for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))

# Then sort the predictions for each user and retrieve the k highest ones.
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:n]

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

9908.0 [12890, 1429, 420, 13721, 44848]
212645.0 [2239, 30531, 15755, 20894, 10256]
228764.0 [13455, 17390, 111, 251, 36891]
156150.0 [10681, 10706, 1871, 16046, 29226]
127429.0 [7012, 41394, 35298, 39, 137]
149084.0 [85, 39, 13786, 971, 16749]
33974.0 [179, 2368, 8648, 4878, 1127]
31556.0 [28161, 8658, 1960, 562, 10876]
201641.0 [24363, 3060, 5895, 13709, 18383]
165232.0 [22630, 16680, 14549, 48570, 38352]
51883.0 [3039, 5514, 2747, 5100, 11635]
126736.0 [11080, 1083, 1295, 28548, 12803]
129074.0 [7169, 11141, 1697, 836, 30265]
40943.0 [9010, 8829, 770, 2995, 10263]
226006.0 [3481, 8118, 4031, 20520, 27099]
52584.0 [89, 1651, 5204, 54636, 50102]
235105.0 [916, 2582, 21185, 11750, 3000]
127914.0 [10879, 3588, 8855, 58844, 58064]
92810.0 [13587, 27886, 20971, 46602, 6343]
36606.0 [3103, 5021, 382, 1476, 6044]
190741.0 [1671, 6479, 16352, 19641, 17540]
163202.0 [28089, 2794, 8112, 8613, 48427]
265889.0 [6081, 1842, 19836, 6089, 15163]
60244.0 [7214, 13975, 3060, 6506, 25136]
252827.0 [10

209516.0 [75, 53782, 46173, 25421, 40901]
252347.0 [12176, 2154, 24731, 28310, 56723]
268833.0 [4137, 22952, 10866, 48523, 39316]
114544.0 [26733, 30231, 5763, 2995, 4184]
13552.0 [1419, 4449, 3717, 4608, 9572]
52350.0 [14203, 1745, 29662, 6800, 15490]
29259.0 [9538, 30231, 8305, 6044, 11819]
172030.0 [424, 9153, 8250, 13934, 7417]
171818.0 [799, 21666, 28443, 18987, 22645]
227520.0 [23012, 14172, 17258, 17515, 15855]
211426.0 [2836, 4593, 3206, 9387, 28605]
59189.0 [41406, 23436, 7835, 139, 26866]
68249.0 [51804, 6735, 29938, 18618, 37670]
27140.0 [43309, 26434, 8141, 48226, 18054]
36256.0 [3237, 26063, 46444, 29771, 11059]
105028.0 [15930, 141, 19710, 803, 3713]
246156.0 [2788, 16448, 1126, 25371, 59196]
172512.0 [4603, 21141, 15381, 46852, 10132]
225199.0 [5270, 16812, 27605, 26499, 17146]
200643.0 [39208, 8982, 465, 9490, 22678]
21031.0 [5087, 18822, 18009, 3000]
87746.0 [6420, 34316, 2167, 9153, 4468]
217986.0 [4442, 4493, 40987, 7312, 7318]
125039.0 [2872, 42610, 41955, 1374, 479

## Recommendations for user 33974.0 [179, 2368, 8648, 4878, 1127]]

In [123]:
def recommended_books(id_list):
    recommended_books = []
    ratings = []
    for bookid in id_list:
        isbn = df_sample[df_sample['bookId'] == bookid]['itemID'].unique()
        recommended_books.append(df[df['ISBN'] == isbn[0]]['Book-Title'].unique()[0])
        ratings.append(df[df['ISBN'] == isbn[0]]['Book-Rating'].unique()[0])
    print(recommended_books)
    print(ratings) 

In [124]:
print("Top 5 Books recommended for user 33974.0 are:\n")
recommended_books([179, 2368, 8648, 4878, 1127])

Top 5 Books recommended for user 33974.0 are:

['The Red Tent (Bestselling Backlist)', "Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson", 'Miss Julia Speaks Her Mind : A Novel', 'Lake Wobegon Days (Wisconsin)', 'House of Sand and Fog']
[8.0, 10.0, 0.0, 0.0, 0.0]


# Review the results

bu, bi are scalars representing the biases of user u on item i.
pu, qi are the matrix factorization of the model.

In [95]:
bi = algo_SVD.bi
bi

array([-0.16739787, -0.43292507, -0.16128056, ..., -0.12136825,
       -0.12343032, -0.08600856])

In [96]:
len(bi)

45372

In [97]:
bu = algo_SVD.bu
bu

array([ 0.26342324, -1.55469952,  0.00802471, ...,  0.89112702,
        1.0813883 ,  0.47654428])

In [98]:
len(bu)

1279

In [99]:
pu = algo_SVD.pu
pu

array([[ 0.23198627,  0.10980752, -0.01880436, ..., -0.00786987,
        -0.14368336, -0.08048929],
       [ 0.02654222,  0.55713376, -0.4115174 , ..., -0.29975153,
         0.24122545,  0.25938226],
       [-0.22632518, -0.16505607,  0.23941805, ..., -0.20711527,
        -0.04168391,  0.09290496],
       ...,
       [ 0.26332705,  0.25428495,  0.18228623, ..., -0.48631948,
        -0.06511455,  0.16608321],
       [ 0.43515791,  0.14742412, -0.35561921, ...,  0.32197552,
         0.37894368,  0.29238792],
       [-0.03446867, -0.17069444,  0.14806053, ...,  0.01719836,
         0.05388053,  0.24414265]])

In [100]:
len(pu)

1279

In [101]:
qi = algo_SVD.qi
qi

array([[-0.02677779,  0.01382236,  0.0373636 , ..., -0.04317749,
        -0.00119158, -0.00173599],
       [ 0.04398468, -0.06665709,  0.10620849, ..., -0.01423376,
         0.01510101,  0.03543376],
       [ 0.12296887,  0.14509572,  0.06621548, ..., -0.01540582,
        -0.00946445, -0.03217329],
       ...,
       [-0.01971823, -0.08425023, -0.14418412, ..., -0.02780725,
         0.10579067, -0.10987766],
       [-0.08935417,  0.05661183,  0.03856759, ...,  0.22430715,
        -0.10078336,  0.02159122],
       [ 0.11946452, -0.09216209, -0.11560877, ...,  0.1416677 ,
         0.04868914,  0.1289036 ]])

In [102]:
len(qi)

45372

In [103]:
n_epochs = algo_SVD.n_epochs
n_epochs

20

In [104]:
n_factors = algo_SVD.n_factors
n_factors

100

In [105]:
# Compute and print Mean Squared Error
accuracy.mse(predictions, verbose=True)

MSE: 10.3113


10.311298887593136

In [106]:
# Compute and print Root Mean Squared Error
accuracy.rmse(predictions, verbose=True)

RMSE: 3.2111


3.2111211262724324

In [107]:
# Compute and print Mean Absolute error
accuracy.mae(predictions, verbose=True)

MAE:  2.4268


2.426845915650027

Root Mean Square Error and Mean Absolute Error are low.  This means that the Recommender has good accuracy.

# Precision@k and Recall@k


Precision@k=|TP| / |TP+FP|=|{Recommended items that are relevant}| / |{Recommended items}|(1)

Recall@k=|TP| / |TP+FN|=|{Recommended items that are relevant}| / |{Relevant items}|(2)

In [108]:
# Return precision and recall at k metrics for each user
def precision_recall_at_k(predictions, k = 10, threshold = 4):
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
        
    return precisions, recalls;

In [109]:
precision_list = []
recall_list = []
f1_score_list = []

print("Precision, Recall, F1 score for top K recommendations")
for k_val in range(1, 20):
    # Get precision and recall at k metrics for each user
    precisions, recalls = precision_recall_at_k(predictions, k=k_val)
            
    # Precision and recall can then be averaged over all users
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    f1_score = 2 * (precision * recall) / (precision + recall)
            
    # Save measures
    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1_score)

    print('K:', k_val, '- Precision:', precision, ', Recall:', recall, ', F1 score:', f1_score)

Precision, Recall, F1 score for top K recommendations
K: 1 - Precision: 0.8444096950742768 , Recall: 0.11112122919556064 , F1 score: 0.1963972926004528
K: 2 - Precision: 0.8385457388584832 , Recall: 0.12789396368279424 , F1 score: 0.22193818815581653
K: 3 - Precision: 0.8399791503778997 , Recall: 0.14352602496248393 , F1 score: 0.24516163519603235
K: 4 - Precision: 0.8395230648944488 , Recall: 0.15658315744937734 , F1 score: 0.26393806062859204
K: 5 - Precision: 0.8410867865519938 , Recall: 0.1701741608263809 , F1 score: 0.283074785899131
K: 6 - Precision: 0.8421553296846493 , Recall: 0.18227505190405446 , F1 score: 0.2996863607100155
K: 7 - Precision: 0.8420250195465204 , Recall: 0.19313303207625973 , F1 score: 0.3141990633298043
K: 8 - Precision: 0.8419831341449795 , Recall: 0.20211156859409146 , F1 score: 0.3259752808349213
K: 9 - Precision: 0.8415379078397066 , Recall: 0.20947691472286672 , F1 score: 0.3354524803499671
K: 10 - Precision: 0.842067835734763 , Recall: 0.21677420838149

# Finalize code

Save the model.

In [110]:
import pickle
s = pickle.dumps(algo_SVD)
from joblib import dump, load
dump(algo_SVD, 'models/model_book_recommend.joblib') 

['models/model_book_recommend.joblib']

# Finalize documentation

In [111]:
results_bi_df = pd.DataFrame(bi)
results_bi_df.rename(columns = {0:'bi'}, inplace = True)
results_bi_df.head()

,bi
0,-0.167398
1,-0.432925
2,-0.161281
3,0.575538
4,0.140441


In [112]:
results_bi_df.shape

(45372, 1)

In [113]:
results_bi_df.to_csv('models/model_results_bi.csv')

In [114]:
results_qi_df = pd.DataFrame(qi)
results_qi_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.026778,0.013822,0.037364,0.201676,-0.080588,0.084557,-0.163084,0.066899,-0.114971,0.024658,...,0.112077,-0.010231,0.135827,0.101714,0.157429,0.082421,0.051533,-0.043177,-0.001192,-0.001736
1,0.043985,-0.066657,0.106208,-0.256056,-0.104421,0.051457,0.164441,-0.086338,-0.000269,-0.087802,...,0.049885,-0.144906,-0.206810,-0.015768,-0.106876,0.093815,0.108869,-0.014234,0.015101,0.035434
2,0.122969,0.145096,0.066215,0.062438,-0.070354,-0.140630,-0.135547,0.127880,-0.140781,0.068496,...,-0.051597,0.091758,-0.099374,0.202842,-0.063094,-0.004607,-0.129006,-0.015406,-0.009464,-0.032173
3,0.220078,0.118455,-0.317407,-0.133477,-0.035011,-0.063740,-0.230791,-0.150598,-0.148231,0.151506,...,-0.123070,0.020520,0.074694,0.007702,0.041144,-0.037293,-0.004967,0.073613,-0.071858,0.175186
4,-0.063912,-0.126715,0.147592,-0.022108,0.129107,-0.129638,0.059723,0.047761,0.139974,0.090011,...,-0.106153,-0.029330,0.290057,0.080643,-0.176269,-0.263982,-0.035307,-0.088096,0.094258,-0.083010


In [115]:
results_qi_df.shape

(45372, 100)

In [116]:
results_qi_df.to_csv('models/model_results_qi.csv')

In [117]:
results_bu_df = pd.DataFrame(bu)
results_bu_df.rename(columns = {0:'bu'}, inplace = True)
results_bu_df.head()

,bu
0,0.263423
1,-1.554700
2,0.008025
3,1.766051
4,0.121904


In [118]:
results_bu_df.shape

(1279, 1)

In [119]:
results_bu_df.to_csv('models/model_results_bu.csv')

In [120]:
results_pu_df = pd.DataFrame(pu)
results_pu_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.231986,0.109808,-0.018804,-0.587196,0.209904,-0.119792,0.224158,-0.114433,0.473475,-0.073259,...,-0.049718,-0.105309,-0.239416,0.010704,-0.029869,0.101104,-0.031570,-0.007870,-0.143683,-0.080489
1,0.026542,0.557134,-0.411517,-0.296431,-0.632997,-0.421202,0.248108,-0.200149,-1.007563,-0.411547,...,1.054022,0.026116,0.761679,0.146989,0.351407,-1.052157,-0.072602,-0.299752,0.241225,0.259382
2,-0.226325,-0.165056,0.239418,-0.387156,0.268709,0.470874,-0.014454,-0.291258,0.603157,-0.054168,...,0.165931,-0.114675,0.313213,-0.237895,0.104607,0.193638,0.215916,-0.207115,-0.041684,0.092905
3,-0.096515,0.872174,-0.963919,-0.489458,-0.269778,-0.193771,-0.587157,-0.986400,-0.021041,-0.006131,...,-0.450419,-0.247536,0.233525,-0.393908,-0.097882,-0.010879,0.279301,0.242052,0.423956,0.476640
4,0.535999,0.455841,-0.228031,0.786310,-0.198232,0.146874,-0.480986,0.713713,0.153400,0.341976,...,-0.046635,0.598821,-0.617339,-0.259903,1.053936,0.577133,-0.127137,0.320983,0.409139,-0.146658


In [121]:
results_pu_df.shape

(1279, 100)

In [122]:
results_pu_df.to_csv('models/model_results_pu.csv')